<a href="https://colab.research.google.com/github/AtaruOhto/pytorch_learning/blob/master/001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

class SampleData:
  """
    データ: X, y
    オリジナルの重み: original_weight
    を準備する。
  """
  def __init__(self):
    # 元々の重み
    original_weight = torch.Tensor([1, 2, 3])
    # データ形式: torch.Size([3])

    # Xのデータ準備 
    # データ形式: torch.Size([100, 3])
    X = torch.cat([torch.ones(100, 1), torch.randn(100, 2)], 1)

    # データと重みの内積を計算する。
    dot = torch.mv(X, original_weight)
    
    # 内積に乱数を足してyとする。
    # データ形式: torch.Size([100])
    y = dot + torch.randn(100) * 0.5    

    self.X = X
    self.y = y
    self.original_weight = original_weight

def exec_gradient_descent(weight, X, y):
  # 学習率を0.1とする
  eta = 0.1

  # 損失関数のログ (下のループ中で誤差を格納していく)
  losses = []

  # 50回ループさせて、勾配降下法でパラメータを最適化する
  for epoc in range(50):
    # 前のループ中のbackward()メソッドで計算された勾配の値を削除
    weight.grad = None
    
    # yの予測値を計算する
    y_pred = torch.mv(X, weight)

    # 平均二乗誤差Mean Square Error を計算 (実際の値と予測値のズレがどれだけあるかの指標のひとつ)
    # 誤差が小さいほどモデルの性能が高い。
    loss = torch.mean((y - y_pred) ** 2)

    # 誤差逆伝播して、勾配を計算。
    # weightのrequires_grad=Trueにしているので、自動で微分が計算される。
    loss.backward()

    # 重みを更新する。
    weight.data = weight.data - eta * weight.grad.data

    # ループの終わりにlossを格納する。誤差が小さくなっていくのを確認。
    losses.append(loss.item())
  return weight

data = SampleData()

# Tensorを乱数で初期化して、重みとする。
# (以下でこの重みをループ中で、「実際のyと予測値y_predとの誤差を小さくするように」更新していく。)
# requires_gradをTrueにすることで重みの自動微分を有効にする。
# https://pytorch.org/docs/stable/notes/autograd.html#requires-grad
weight_will_updated = torch.randn(3, requires_grad=True)

# 乱数で作った重み (weight_will_updated) を更新する。
# (Xの計算式から、yの値を正しく予測するように重みを更新する。)
weight = exec_gradient_descent(weight_will_updated, data.X, data.y)

# matplotlibで誤差 (loss) の推移を描画する。
%matplotlib inline
from matplotlib import pyplot as plt
plt.plot(losses)

# 元々の重みと、更新された重みの差分を見る。
# 差分は極小になっている。
print(torch.abs(data.original_weight - weight))